<a href="https://colab.research.google.com/github/kingsuching/data-301-final-project/blob/main/Data_Collection_%26_Cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import requests
import numpy as np
import sklearn
apiKey = "ZjIzO4zf314rMCj1CpTGMNyPkJLlgGX1"
url = "https://aeroapi.flightaware.com/aeroapi"
headers = {"x-apikey": apiKey}
payload = {"max_pages": 2}
airport = "KSFO"
def api(url1, pages = 2):
  payload = {"max_pages": pages}
  return requests.get(url + url1, headers=headers, params=payload).json()

In [ ]:
airports = ["KSFO", "KOAK", "KSJC"] # include bay area airports to train models
df_departures = pd.DataFrame()
def query(airport):
  request = api("/airports/%s/flights/departures" % airport)
  try:
    data = pd.json_normalize(request, "departures")
    return data
  except:
    return None
for airport in airports:
  data = query(airport)
  while data is None:
    data = query(airport)
  df_departures = df_departures.append(data)

In [ ]:
request = api("/airports/delays", 10)
df_delays = pd.json_normalize(request["delays"], meta="reasons")
df_delays["delay_secs"] = df_delays["delay_secs"]/60 # convert to minutes
df_delays.rename(columns={"delay_secs": "delay_min"}, inplace=True)
cpy = pd.DataFrame()
for i in range(len(df_delays)):
  data = pd.DataFrame(df_delays.loc[i, "reasons"])
  data["airport"] = df_delays.loc[i, "airport"]
  cpy = cpy.append(data)
df_delays = cpy
df_delays.index = range(len(df_delays))

In [ ]:
import time
df = pd.DataFrame()
def query(airport):
  request = api("/airports/%s/weather/observations" % airport)
  try:
    data = pd.json_normalize(request, "observations")
    return data
  except:
    return None

for airport in airports:
  request = api("/airports/%s/weather/observations" % airport)
  data = query(airport)
  while data is None:
    data = query(airport)
  df = df.append(data)

In [ ]:
from re import A
from sklearn.preprocessing import *
from sklearn.model_selection import *
from sklearn.compose import *
from sklearn.pipeline import *
from sklearn.neighbors import *
from sklearn.compose import *
from sklearn.cluster import *
means = df_departures.groupby("origin.code")["departure_delay"].mean()
arr = []
for airport in df["airport_code"]:
  arr.append(means[airport])
df["delay_secs"] = arr

In [ ]:
d = pd.DataFrame()
for i in df["clouds"]:
  d = d.append(pd.DataFrame(i).loc[0])
d.index = range(len(d))
df[d.columns] = d
df = df.drop(["clouds"], axis=1)

In [ ]:
df_arrivals = pd.DataFrame()
def query(airport):
  try:
    return pd.json_normalize(api("/airports/%s/flights/scheduled_arrivals" % airport, 5), "scheduled_arrivals")
  except:
    return None
for airport in airports:
  data = query(airport)
  while data is None:
    data = query(airport)
  df_arrivals = df_arrivals.append(data)

In [ ]:
df.head(1)

,airport_code,cloud_friendly,conditions,pressure,pressure_units,raw_data,temp_air,temp_dewpoint,temp_perceived,relative_humidity,...,visibility_units,wind_direction,wind_friendly,wind_speed,wind_speed_gust,wind_units,delay_secs,altitude,symbol,type
0,KSFO,Partly cloudy,None,30.25,in Hg,KSFO 142056Z 06004KT 10SM BKN200 15/08 A3025 R...,15,8,15,65,...,SM,60,Light winds,4,0,KT,51.833333,20000.0,BKN200,BKN


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

In [ ]:
df_arrivals.head(1)

,ident,fa_flight_id,operator,operator_iata,flight_number,registration,atc_ident,inbound_fa_flight_id,codeshares,blocked,...,seats_cabin_first,gate_origin,gate_destination,terminal_origin,terminal_destination,type,origin.code,origin.airport_info_url,destination.code,destination.airport_info_url
0,SKW4671,SKW4671-1647119099-airline-0302,SKW,OO,4671,N652BR,None,SKW5405-1647107400-schedule-0031,[UAL4671],False,...,NaN,4,E3,None,3,Airline,KBFL,/airports/KBFL,KSFO,/airports/KSFO


In [ ]:
df_arrivals.columns

Index(['ident', 'fa_flight_id', 'operator', 'operator_iata', 'flight_number',
       'registration', 'atc_ident', 'inbound_fa_flight_id', 'codeshares',
       'blocked', 'diverted', 'cancelled', 'position_only', 'departure_delay',
       'arrival_delay', 'filed_ete', 'scheduled_out', 'estimated_out',
       'actual_out', 'scheduled_off', 'estimated_off', 'actual_off',
       'scheduled_on', 'estimated_on', 'actual_on', 'scheduled_in',
       'estimated_in', 'actual_in', 'progress_percent', 'status',
       'aircraft_type', 'route_distance', 'filed_airspeed', 'filed_altitude',
       'route', 'baggage_claim', 'seats_cabin_business', 'seats_cabin_coach',
       'seats_cabin_first', 'gate_origin', 'gate_destination',
       'terminal_origin', 'terminal_destination', 'type', 'origin.code',
       'origin.airport_info_url', 'destination.code',
       'destination.airport_info_url'],
      dtype='object')

In [ ]:
df_departures.head(1)

,ident,fa_flight_id,operator,operator_iata,flight_number,registration,atc_ident,inbound_fa_flight_id,codeshares,blocked,...,gate_origin,gate_destination,terminal_origin,terminal_destination,type,origin.code,origin.airport_info_url,destination.code,destination.airport_info_url,destination
0,SKW3352,SKW3352-1647120684-airline-0074,SKW,OO,3352,N405SY,None,SKW3459-1647112887-airline-0134,"[ASA3352, JAL5460, SIA1408]",False,...,D3,24,2,2,Airline,KSFO,/airports/KSFO,KSAN,/airports/KSAN,NaN


In [ ]:
df_departures.columns

Index(['ident', 'fa_flight_id', 'operator', 'operator_iata', 'flight_number',
       'registration', 'atc_ident', 'inbound_fa_flight_id', 'codeshares',
       'blocked', 'diverted', 'cancelled', 'position_only', 'departure_delay',
       'arrival_delay', 'filed_ete', 'scheduled_out', 'estimated_out',
       'actual_out', 'scheduled_off', 'estimated_off', 'actual_off',
       'scheduled_on', 'estimated_on', 'actual_on', 'scheduled_in',
       'estimated_in', 'actual_in', 'progress_percent', 'status',
       'aircraft_type', 'route_distance', 'filed_airspeed', 'filed_altitude',
       'route', 'baggage_claim', 'seats_cabin_business', 'seats_cabin_coach',
       'seats_cabin_first', 'gate_origin', 'gate_destination',
       'terminal_origin', 'terminal_destination', 'type', 'origin.code',
       'origin.airport_info_url', 'destination.code',
       'destination.airport_info_url', 'destination'],
      dtype='object')

In [ ]:
df.to_csv("drive/MyDrive/df.csv")
df_departures.to_csv("drive/MyDrive/df_departures.csv")
df_arrivals.to_csv("drive/MyDrive/df_arrivals.csv")

FileNotFoundError: ignored